In [1]:
import pandas as pd
import json
from tqdm import tqdm
import os

## Online data

### Social media data

In [2]:
posts_data = pd.read_csv("../data/fb_data_with_predictions.csv")

## Offline data

In [3]:
offline_data = pd.read_csv("../data/overall_nlrb_all2024.csv")

/var/folders/3g/c8qm42k164b6t37cxylyjjfc0000gn/T/ipykernel_15475/3571609864.py:1: DtypeWarning: Columns (17,18,25,26,27,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  offline_data = pd.read_csv("../data/overall_nlrb_all2024.csv")


## Map between social media accounts and union names

In [6]:
highlevel_unions = pd.read_csv("../../../data/fb_data/fb_highlevel_unions_tocheck_v4.csv")
highlevel_unions = highlevel_unions.dropna(subset=["account_match"])

In [7]:
highlevel_unions_handles = highlevel_unions[["union", "account_match"]]
highlevel_unions_handles.columns = ["main_union", "handle"]

Add info on main_union (if available) from hierarchy file

In [9]:
list_files = []
for file in os.listdir("../../../data/"):
    if file.endswith(".json") and "hierarchy_unions" in file:
        v_number = int(file.split("_")[-1].split(".")[0].split("v")[-1])
        list_files.append([file, v_number])

# sort files by version number
list_files.sort(key=lambda x: x[1])

file = list_files[-1]

with open("../../../data/" + file[0], "r") as f:
    union_names = json.load(f)

From the hierarchy, we have to find all unions that have union name or main_union equal to union

In [10]:
union_dict = {}
for union in union_names:
    if union_names[union]["main_union"] in highlevel_unions_handles["main_union"].values:
        if union_names[union]["main_union"] not in union_dict:
            union_dict[union_names[union]["main_union"]] = []
        union_dict[union_names[union]["main_union"]].append(union)

In [11]:
rows = []
for main_union in union_dict:
    for union in union_dict[main_union]:
        handle = highlevel_unions_handles[highlevel_unions_handles["main_union"] == main_union]["handle"].values[0]
        rows.append([main_union, union, handle])

In [12]:
unions_df = pd.DataFrame(rows, columns=["main_union", "union", "handle"])

## Offline-online map

Offline events dates

In [13]:
offline_data["Labor Org 1 Name"] = offline_data["Labor Org 1 Name"].str.lower()
offline_data["Labor Org 2 Name"] = offline_data["Labor Org 2 Name"].str.lower()
offline_data["Labor Org 3 Name"] = offline_data["Labor Org 3 Name"].str.lower()
offline_data["Labor Org3"] = offline_data["Labor Org3"].str.lower()
offline_data["Union To Certify"] = offline_data["Union To Certify"].str.lower()

In [15]:
# Create a dictionary for union handles to avoid repeated DataFrame lookups
union_handles = dict(zip(unions_df["union"], unions_df["handle"]))

# Filter offline data once for "RC" and "RD" cases
rc_cases = offline_data[offline_data["Case Number"].str.contains("RC")]

rc_cases = rc_cases.rename(columns={"City_x": "City"})

rc_cases['Date Filed'] = pd.to_datetime(rc_cases['Date Filed']).dt.date
rc_cases['Election Held Date'] = pd.to_datetime(rc_cases['Election Held Date'], errors='coerce', format='mixed').dt.date
rc_cases["Closed Date"] = pd.to_datetime(rc_cases["Closed Date"], errors='coerce', format='mixed').dt.date

In [16]:
events_dict_complete = {}

for i, row in tqdm(unions_df.iterrows(), total=unions_df.shape[0]):
    union = row["union"]

    main_union = row["main_union"]
    handle = union_handles[union]

    if main_union not in events_dict_complete:
        events_dict_complete[main_union] = {
            "handle": handle,
            "cases": []
        }


    # Filter rows where union is either in "Union To Certify" or "Labor Org 1 Name"
    rc_union_rows = rc_cases[(rc_cases["Union To Certify"] == union)]
    # Filter rows where union is in the election
    rc_union_rows = pd.concat([rc_union_rows, rc_cases[(rc_cases["Labor Org 1 Name"] == union) & (pd.isnull(rc_cases["Union To Certify"]))]])
    rc_union_rows = pd.concat([rc_union_rows, rc_cases[(rc_cases["Labor Org 2 Name"] == union)]])
    rc_union_rows = pd.concat([rc_union_rows, rc_cases[(rc_cases["Labor Org 3 Name"] == union)]])
    rc_union_rows = pd.concat([rc_union_rows, rc_cases[(rc_cases["Labor Org3"] == union)]])

    for _, row in rc_union_rows.iterrows():
        case_number = row["Case Number"]

        election_date_indata = True

        if ((row["Labor Org 1 Name"] == union) & (pd.isnull(row["Union To Certify"]))) or row["Union To Certify"] == union:
            if row["Cert of Rep (Win)"] == "WON" or row["Certification of Representative Date (Win)"] == "WON":
                if not pd.isnull(row["Election Held Date"]):
                    case_winning_election_date = row["Election Held Date"]
                else:
                    case_winning_election_date = row["Closed Date"]
                    election_date_indata = False
            else:
                case_winning_election_date = None

            if row["Cert of Results (Loss)"] == "LOSS" or row["Certification of Results Date (Loss)"] == "LOSS":
                if not pd.isnull(row["Election Held Date"]):
                    case_losing_election_date = row["Election Held Date"]
                else:
                    case_losing_election_date = row["Closed Date"]
                    election_date_indata = False
            else:
                case_losing_election_date = None
        else:
            case_winning_election_date = None
            if not pd.isnull(row["Election Held Date"]):
                case_losing_election_date = row["Election Held Date"]
            else:
                case_losing_election_date = row["Closed Date"]
                election_date_indata = False

        events_dict_complete[main_union]["cases"].append({
            "union": union,
            "case_number": case_number,
            # convert to string to avoid json serialization error
            "case_winning_election_date":  str(case_winning_election_date),
            "case_losing_election_date": str(case_losing_election_date),
        })


100%|██████████| 5518/5518 [00:47<00:00, 115.13it/s]


In [ ]:
# save events dict to json
with open("../data/events_dict_fb_all2024.json", "w") as f:
    json.dump(events_dict_complete, f, indent=4)